<a href="https://colab.research.google.com/github/Pathway2008/income/blob/main/drop_2col.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/income/train.csv')
test = pd.read_csv('/content/drive/MyDrive/income/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/income/sample_submission.csv')

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      20000 non-null  object
 1   Age                     20000 non-null  int64 
 2   Gender                  20000 non-null  object
 3   Education_Status        20000 non-null  object
 4   Employment_Status       20000 non-null  object
 5   Working_Week (Yearly)   20000 non-null  int64 
 6   Industry_Status         20000 non-null  object
 7   Occupation_Status       20000 non-null  object
 8   Race                    20000 non-null  object
 9   Hispanic_Origin         20000 non-null  object
 10  Martial_Status          20000 non-null  object
 11  Household_Status        20000 non-null  object
 12  Household_Summary       20000 non-null  object
 13  Citizenship             20000 non-null  object
 14  Birth_Country           20000 non-null  object
 15  Bi

In [14]:
train.drop(['ID'],axis=1,inplace=True)
test.drop(['ID'],axis=1,inplace=True)

In [15]:
train['Invest_income'] = train[['Gains','Dividends']].sum(axis=1) - train['Losses']
test['Invest_income'] = test[['Gains','Dividends']].sum(axis=1) - test['Losses']

In [16]:
train_except = train[
    (
        (train['Education_Status'] != 'Children') &
        (train['Employment_Status'] != 'Not Working') &
        (train['Industry_Status'] != 'Not in universe or children') &
        (train['Industry_Status'] != 'Armed Forces') &
        (train['Occupation_Status'] != 'Armed Forces') &
        (train['Occupation_Status'] != 'Unknown')
    )]

In [17]:
test_except = test[
    (
        (test['Education_Status'] != 'Children') &
        (test['Employment_Status'] != 'Not Working') &
        (test['Industry_Status'] != 'Not in universe or children') &
        (test['Industry_Status'] != 'Armed Forces') &
        (test['Occupation_Status'] != 'Armed Forces') &
        (test['Occupation_Status'] != 'Unknown')
    )]

In [18]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
object_columns = train_except.select_dtypes(include='object')
for feature in object_columns.columns:
    le = LabelEncoder()

    # Convert the column to string type
    train_except[feature] = train_except[feature].astype(str)
    le.fit(train_except[feature])
    train_except[feature] = le.transform(train_except[feature])

    # Make sure the test set has the same data type
    test_except[feature] = test_except[feature].astype(str)

    # Handle unseen labels in the test set
    for label in np.unique(test_except[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)

    test_except[feature] = le.transform(test_except[feature])

<ipython-input-18-828966699332>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_except[feature] = train_except[feature].astype(str)
<ipython-input-18-828966699332>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_except[feature] = le.transform(train_except[feature])
<ipython-input-18-828966699332>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [19]:
train_except.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15281 entries, 0 to 19999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   Age                     15281 non-null  int64
 1   Gender                  15281 non-null  int64
 2   Education_Status        15281 non-null  int64
 3   Employment_Status       15281 non-null  int64
 4   Working_Week (Yearly)   15281 non-null  int64
 5   Industry_Status         15281 non-null  int64
 6   Occupation_Status       15281 non-null  int64
 7   Race                    15281 non-null  int64
 8   Hispanic_Origin         15281 non-null  int64
 9   Martial_Status          15281 non-null  int64
 10  Household_Status        15281 non-null  int64
 11  Household_Summary       15281 non-null  int64
 12  Citizenship             15281 non-null  int64
 13  Birth_Country           15281 non-null  int64
 14  Birth_Country (Father)  15281 non-null  int64
 15  Birth_Country (Mother)  

In [6]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 7.4 MB/s eta 0:00:00


In [7]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from math import sqrt
from sklearn.metrics import make_scorer, mean_squared_error

In [20]:
from sklearn.model_selection import train_test_split
X = train_except.drop(['Income'], axis=1)
y = train_except['Income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 43)

In [21]:
from sklearn.model_selection import KFold
kf = StratifiedKFold(n_splits=5, shuffle=False)

In [22]:
def rmse_scorer(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

# Random Forest
rf_model = RandomForestRegressor(random_state=43)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_scores = cross_val_score(rf_model, X_train, y_train, cv=kf, scoring=make_scorer(rmse_scorer))
rf_rmse = rmse_scorer(y_test, rf_predictions)


# Extra Trees
et_model = ExtraTreesRegressor(random_state=43)
et_model.fit(X_train, y_train)
et_predictions = et_model.predict(X_test)
et_scores = cross_val_score(et_model, X_train, y_train, cv=kf, scoring=make_scorer(rmse_scorer))
et_rmse = rmse_scorer(y_test, et_predictions)

#xgb
xgb_model = xgb.XGBRegressor(random_state=43)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)
xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=kf, scoring=make_scorer(rmse_scorer))
xgb_rmse = rmse_scorer(y_test, xgb_predictions)

#cat
cat_model = CatBoostRegressor(random_seed=43)
cat_model.fit(X_train, y_train)
cat_predictions = cat_model.predict(X_test)
cat_scores = cross_val_score(cat_model, X_train, y_train, cv=kf, scoring=make_scorer(rmse_scorer))
cat_rmse = rmse_scorer(y_test, cat_predictions)

#lgb
lgb_model = LGBMRegressor(random_state=43)
lgb_model.fit(X_train, y_train)
lgb_predictions = lgb_model.predict(X_test)
lgb_scores = cross_val_score(lgb_model, X_train, y_train, cv=kf, scoring=make_scorer(rmse_scorer))
lgb_rmse = rmse_scorer(y_test, lgb_predictions)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Learning rate set to 0.059538
0:	learn: 725.3869738	total: 51.5ms	remaining: 51.5s
1:	learn: 723.4089930	total: 54.9ms	remaining: 27.4s
2:	learn: 721.4452983	total: 60ms	remaining: 19.9s
3:	learn: 719.2878376	total: 64.9ms	remaining: 16.1s
4:	learn: 717.3985938	total: 68.1ms	remaining: 13.6s
5:	learn: 715.9774984	total: 71.2ms	remaining: 11.8s
6:	learn: 714.6620422	total: 74.1ms	remaining: 10.5s
7:	learn: 713.5644797	total: 77.3ms	remaining: 9.58s
8:	learn: 712.0961755	total: 80.9ms	remaining: 8.91s
9:	learn: 710.7397483	total: 84.1ms	remaining: 8.33s
10:	learn: 709.6277230	total: 87.3ms	remaining: 7.85s
11:	learn: 708.6354272	total: 90.6ms	remaining: 7.46s
12:	learn: 707.7110092	total: 94.3ms	remaining: 7.16s
13:	learn: 706.5615258	total: 97.8ms	remaining: 6.88s
14:	learn: 705.7750883	total: 101ms	remaining: 6.66s
15:	learn: 704.6131175	total: 105ms	remaining: 6.46s
16:	learn: 703.7874761	total: 108ms	remaining: 6.26s
17:	learn: 702.9173393	total: 112ms	remaining: 6.1s
18:	learn: 702.

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
11:	learn: 713.4964628	total: 97.1ms	remaining: 8s
12:	learn: 712.5135346	total: 106ms	remaining: 8.04s
13:	learn: 711.6811343	total: 112ms	remaining: 7.86s
14:	learn: 710.4850220	total: 116ms	remaining: 7.6s
15:	learn: 709.4620057	total: 120ms	remaining: 7.38s
16:	learn: 708.6680620	total: 129ms	remaining: 7.44s
17:	learn: 707.4420545	total: 137ms	remaining: 7.48s
18:	learn: 706.6389367	total: 152ms	remaining: 7.86s
19:	learn: 705.6359349	total: 161ms	remaining: 7.87s
20:	learn: 704.8751464	total: 167ms	remaining: 7.79s
21:	learn: 704.0524830	total: 178ms	remaining: 7.93s
22:	learn: 703.2421577	total: 185ms	remaining: 7.84s
23:	learn: 702.6008752	total: 194ms	remaining: 7.9s
24:	learn: 701.8494950	total: 203ms	remaining: 7.93s
25:	learn: 701.2776416	total: 213ms	remaining: 7.98s
26:	learn: 700.4085754	total: 225ms	remaining: 8.1s
27:	learn: 699.8361279	total: 231ms	remaining: 8.03s
28:	learn: 699.2132765	total: 235ms	remaining: 7.86s
29:	learn: 698.

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001382 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 823
[LightGBM] [Info] Number of data points in the train set: 8556, number of used features: 22
[LightGBM] [Info] Start training from score 729.751286
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002438 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 821
[LightGBM] [Info] Number of data points in the train set: 8557, number of used features: 22
[LightGBM] [Info] Start training from score 726.718944
[LightGBM] [Warning] Found w

In [ ]:
from sklearn.ensemble import VotingRegressor
voting_regressor = VotingRegressor([('xgb', xgb_model), ('catboost', cat_model), ('et',et_model),('rf',rf_model),('lgb',lgb_model)])#cat_model로 수정필요
voting_regressor.fit(X_train.drop(['Losses','Dividends'],axis=1), y_train)

# voting_predictions = voting_regressor.predict(X_test.drop(['Losses','Dividends'],axis=1))

# voting_rmse = rmse_scorer(y_test, voting_predictions)
# voting_scores = cross_val_score(voting_regressor, X_train, y_train, cv=kf, scoring=make_scorer(rmse_scorer))

In [24]:
voting_predictions = voting_regressor.predict(test_except.drop(['Losses','Dividends'],axis=1))
len(voting_predictions), len(test_except), len(test)

(7672, 7672, 10000)

In [ ]:
test_except['except'] = voting_predictions

In [30]:
test_except.head()

,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,Martial_Status,...,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Invest_income,except
2,18,0,8,0,52,17,10,4,0,5,...,39,39,39,5,0,0,0,1,0,444.305179
3,39,0,1,2,30,11,10,4,0,1,...,39,39,39,2,0,0,0,2,0,773.362953
5,31,1,7,2,24,3,12,4,0,1,...,39,39,39,2,0,0,0,2,0,824.924748
6,17,1,7,2,51,17,10,4,0,5,...,39,39,39,5,0,0,0,1,0,429.959448
9,43,1,3,2,52,19,0,1,0,1,...,39,39,39,2,0,0,0,2,0,1220.451934


In [36]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10000 non-null  object
 1   Income  10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [35]:
sub = pd.read_csv('/content/drive/MyDrive/income/sample_submission.csv')

In [37]:
for idx in test_except.index:
    sub.at[idx, 'Income'] = test_except.at[idx, 'except']

In [38]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      10000 non-null  object 
 1   Income  10000 non-null  float64
dtypes: float64(1), object(1)
memory usage: 156.4+ KB


In [40]:
sub.to_csv('Loss,Dividend_drop.csv',index=False)